In [2]:
import yfinance as yf
import os
import pandas as pd
import numpy as np
from yahoofinancials import YahooFinancials

In [4]:
def fetch_ta_df(value):
    #input value, data from yahoo finance
    #value = input("Please enter the currency: \n")
    
    ticker = yf.Ticker(f'{value}-USD')
    df_ta = ticker.history(period="5y")
    df_ta.drop(['Open', 'High', 'Low', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    #add index value
    p = df_ta.index.values
    df_ta.insert( 0, column="Date",value = p)
    #Calculate RSI
    delta = df_ta['Close'].diff(1) #difference from the previous day
    delta = delta.dropna()
    #get the positive gains(up) and negative gains(down)
    up = delta.copy()
    down = delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    #Get the time period 14 days
    period = 14
    #calculate the average gain and loss
    AVG_Gain = up.rolling(window=period).mean()
    AVG_Loss = abs(down.rolling(window=period).mean())
    #Calculate RSI
    RS = AVG_Gain / AVG_Loss #relative strength
    #RSI
    RSI = 100.0 - (100.0/(1.0 + RS))
    #Put it all together
    new_df = pd.DataFrame()
    new_df['Close'] = df_ta['Close']
    new_df['RSI'] = RSI
    new_df
    #Get Buy and Sell Signals
    conditions = [
        (new_df['RSI'] < 80) & (new_df['RSI'] > 20),
        (new_df['RSI'] < 80),
        (new_df['RSI'] > 20)]
    choices = [0, -2, 2]
    df_ta['signal_rsi'] = np.select(conditions, choices, default='0')
    #Bollinger Bands indicator
    period= 20
    #Simple Moving Average
    df_ta['SMA'] = df_ta['Close'].rolling(window=period).mean()
    #Standard deviation
    df_ta['STD'] = df_ta['Close'].rolling(window=period).std()
    #upper band
    df_ta['Upper'] = df_ta['SMA'] + (df_ta['STD']*2)
    #lower band
    df_ta['Lower'] = df_ta['SMA'] - (df_ta['STD']*2)
    #Create Signal Value
    conditions = [
        (df_ta.Lower > df_ta.Close) & (df_ta.Upper < df_ta.Close),
        (df_ta.Lower > df_ta.Close),
        (df_ta.Upper < df_ta.Close)]
    choices = [0, 2, -2]
    df_ta['signal_bb'] = np.select(conditions, choices, default='0')
    df_ta = df_ta.dropna()
    #On Balance Volume indicator
    df_ta['Daily Returns'] = df_ta['Close'].pct_change()
    df_ta['signal_obv'] = 0
    df_ta.loc[df_ta['Daily Returns'] > 0, 'signal_obv'] = 2
    df_ta.loc[df_ta['Daily Returns'] < 0, 'signal_obv'] = -2
    df_ta.dropna(inplace= True)
    #MVA indicator
    #Create the simple MVA
    def SMA(data, period = 30, column ='Close'):
        return data[column].rolling(window=period).mean()
    #Creating the Exponential MVA
    def EMA(data, period=20, column='Close'):
        return data[column].ewm(span=period, adjust=False).mean()
    #Calculate MVA Convergence/Divergence
    def MACD(data, period_long=28, period_short=12, period_signal=9, column='Close'):
        #calculate the short term exponential MVA
        ShortEMA = EMA(data, period_short, column=column)
        #calculate the long term exponential MVA
        LongEMA = EMA(data, period_long, column=column)
        #calculate the MVA Convergence/Divergence MACD
        data['MACDA'] = ShortEMA - LongEMA
        #calculate the signal line
        data['Signal_Line'] = EMA(data, period_signal, column='MACD')
        return data
    df_ta['SMA'] = SMA(df_ta)
    df_ta['EMA'] = EMA(df_ta)
    #Create a Simple Moving Average indicator
    def SMA(data, period = 30, column = 'Close'):
        return data[column].rolling(window=period).mean()
    #Create two new columns to store 20 and 50 day SMA
    df_ta['SMA20']=SMA(df_ta, 20)
    df_ta['SMA50']=SMA(df_ta, 50)
    #Get Buy and Sell Signals
    df_ta['signal_mva'] = np.where(df_ta['SMA20']> df_ta['SMA50'], 2, -2)
    # Delete redundant data
    df_ta.drop(['SMA', 'STD', 'Upper', 'Lower','Daily Returns', 'EMA','SMA20','SMA50'], axis=1, inplace=True)
    #prepare datatype
    df_ta['signal_rsi'] = df_ta['signal_rsi'].astype('int')
    df_ta['signal_bb'] = df_ta['signal_bb'].astype('int')
    df_ta['signal_mva'] = df_ta['signal_mva'].astype('int')
    df_ta['signal_obv'] = df_ta['signal_obv'].astype('int')
    #add calculated average
    zaehler =  df_ta.apply(lambda x: (0.3 * x['signal_bb']) + (0.3 * x['signal_mva']) + (0.2 * x['signal_obv']) +(0.2 * x['signal_rsi']), axis=1)
    df_ta['calculated_average'] = zaehler / 1
    
    df_ta['ta_query'] = value
    
    #Plot Frame
    return df_ta

In [5]:
currencies = ['ETH2', 'SHIB', 'SOL', 'ETH', 'LINK', 'BTC', 'DOGE']

In [8]:
df_ta_ETH

,Date,Close,Volume,signal_rsi,signal_bb,signal_obv,signal_mva,calculated_average,ta_query
Date,,,,,,,,,
2017-11-29,2017-11-29,427.523010,2675940096,0,0,-2,-2,-1.0,ETH
2017-11-30,2017-11-30,447.114014,1903040000,0,0,2,-2,-0.2,ETH
2017-12-01,2017-12-01,466.540009,1247879936,0,0,2,-2,-0.2,ETH
2017-12-02,2017-12-02,463.449005,943649984,0,0,-2,-2,-1.0,ETH
2017-12-03,2017-12-03,465.852997,990556992,0,0,2,-2,-0.2,ETH
...,...,...,...,...,...,...,...,...,...
2022-01-29,2022-01-29,2597.084717,11172062661,0,0,2,-2,-0.2,ETH
2022-01-30,2022-01-30,2603.466553,9501221177,0,0,2,-2,-0.2,ETH
2022-01-31,2022-01-31,2688.278809,13778234614,0,0,2,-2,-0.2,ETH


In [6]:
df_ta_BTC = fetch_ta_df('BTC')
df_ta_SHIB = fetch_ta_df('SHIB')
df_ta_SOL = fetch_ta_df('SOL')
df_ta_ETH = fetch_ta_df('ETH')
df_ta_LINK = fetch_ta_df('LINK')
df_ta_DOGE = fetch_ta_df('DOGE')
df_ta_SHIB = fetch_ta_df('SHIB')

In [7]:
df_ta_BTC.to_csv('ta_df/df_ta_BTC.csv')
df_ta_SHIB.to_csv('ta_df/df_ta_SHIB.csv')
df_ta_SOL.to_csv('ta_df/df_ta_SOL.csv')
df_ta_ETH.to_csv('ta_df/df_ta_ETH.csv')
df_ta_LINK.to_csv('ta_df/df_ta_LINK.csv')
df_ta_DOGE.to_csv('ta_df/df_ta_DOGE.csv')
df_ta_SHIB.to_csv('ta_df/df_ta_SHIB.csv')